In [3]:
import cohere
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage, SystemMessage
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the Cohere API key from environment variables
cohere_api_key = os.getenv('COHERE_API_KEY')
if not cohere_api_key:
    raise ValueError("Cohere API key not found. Please set it in the .env file.")

cohere_client = cohere.Client(cohere_api_key)

class LLMInterface:
    def query(self, prompt):
        try:
            response = cohere_client.generate(prompt=prompt)
            return response.generations[0].text
        except Exception as e:
            print(f"Error querying Cohere API: {e}")
            return None

class LangChainModel:
    def __init__(self, llm_interface):
        self.llm_interface = llm_interface

    def get_suggestions(self, parameters):
        prompt = self.create_prompt(parameters)
        return self.llm_interface.query(prompt)

    def create_prompt(self, parameters):
        prompt = "Generate suggestions based on the following parameters:\n"
        for param in parameters:
            prompt += f"{param['name']}: {param['description']}\n"
        return prompt

parameters = [
    {"name": "Relevance", "description": "Measures how relevant the suggested application is to current market trends and user needs."},
    {"name": "Profitability", "description": "Estimates the potential revenue that the suggested application could generate."},
    {"name": "Complexity", "description": "Assesses the technical complexity involved in developing the suggested application."},
    {"name": "User Engagement", "description": "Predicts the level of user engagement and retention for the suggested application."},
    {"name": "Development Time", "description": "Estimates the time required to develop the suggested application."}
]

llm_interface = LLMInterface()
langchain_model = LangChainModel(llm_interface)

suggestions = langchain_model.get_suggestions(parameters)
if suggestions:
    print(suggestions)
else:
    print("Failed to get suggestions.")

ValueError: Cohere API key not found. Please set it in the .env file.